In [1]:
import numpy as np
import scipy.sparse as sp
from graphgallery import functional as gf 
import graphgallery as gg
gg.__version__

'0.8.0'

# Use PyTorch backend

In [2]:
gg.set_backend("torch")

PyTorch 1.6.0 Backend

# Load datasets
+ cora
+ citeseer
+ cora_ml
+ pubmed

In [3]:
dataset = 'cora'
data = gg.datasets.NPZDataset(dataset, root='~/GraphData/datasets/', transform="standardize",  verbose=False)
splits = data.split_nodes(random_state=15)
graph = data.graph

# MedianGCN and TrimmedGCN

In [4]:
from median_gcn import MedianGCN
from trimmed_gcn import TrimmedGCN

## You can even import them from graphgallery
## e.g.
# from graphgallery.gallery.nodeclas import MedianGCN
# from graphgallery.gallery.nodeclas import TrimmedGCN

In [5]:
# Median convolution needs a smller `weight_decay`
median = MedianGCN(device="cuda", seed=10).make_data(graph).build(hids=64, weight_decay=5e-5)
# median = TrimmedGCN(device="cuda", seed=10).make_data(graph).build(hids=64, weight_decay=5e-5)
his = median.fit(splits.train_nodes, splits.val_nodes, verbose=1, epochs=200)

# performance on test set
median.evaluate(splits.test_nodes)

Training...
200/200 [====================] - Total: 330.31s - 1s/step - loss: 0.025 - accuracy: 0.992 - val_loss: 1.037 - val_accuracy: 0.815
Testing...
1/1 [====================] - Total: 112.30ms - 112ms/step - loss: 0.516 - accuracy: 0.853


Objects in BunchDict:
╒══════════╤═══════════╕
│ Names    │   Objects │
╞══════════╪═══════════╡
│ loss     │  0.515891 │
├──────────┼───────────┤
│ accuracy │  0.852616 │
╘══════════╧═══════════╛

# Baseline: GCN

In [6]:
gcn = gg.gallery.nodeclas.GCN(device="cuda", seed=10).make_data(graph).build(hids=64)
his = gcn.fit(splits.train_nodes, splits.val_nodes, verbose=1, epochs=100)

# performance on test set
gcn.evaluate(splits.test_nodes)

Training...
100/100 [====================] - Total: 384.03ms - 3ms/step - loss: 0.037 - accuracy: 0.992 - val_loss: 0.621 - val_accuracy: 0.811
Testing...
1/1 [====================] - Total: 1.77ms - 1ms/step - loss: 0.618 - accuracy: 0.850


Objects in BunchDict:
╒══════════╤═══════════╕
│ Names    │   Objects │
╞══════════╪═══════════╡
│ loss     │  0.617904 │
├──────────┼───────────┤
│ accuracy │  0.849598 │
╘══════════╧═══════════╛

# All parameters

In [7]:
print(median.cfg)

backend: PyTorch
boolx: bool
data:
  adj_transform: normalize_adj
  attr_transform: None
  device: None
  edge_transform: None
  graph_transform: None
  label_transform: None
device: cuda
evaluate:
  Progbar:
    width: 20
  cache_test_data: False
  verbose: 1
fit:
  EarlyStopping:
    baseline: None
    enabled: False
    mode: auto
    monitor: val_loss
    patience: 10
    restore_best_weights: True
    verbose: 1
  Logger:
    enabled: False
    filepath: None
    name: None
  ModelCheckpoint:
    enabled: True
    monitor: val_accuracy
    path: ./MedianGCN_checkpoint_95abb3.h5
    remove_weights: True
    save_best_only: True
    save_weights_only: True
    vervose: 0
  Progbar:
    width: 20
  TensorBoard:
    enabled: False
    histogram_freq: 0
    log_dir: ./logs
    update_freq: epoch
    write_graph: True
    write_images: True
  TerminateOnNaN:
    enabled: False
  cache_train_data: False
  cache_val_data: False
  epochs: 200
  verbose: 1
floatx: float32
intx: int64
model:

In [8]:
print(gcn.cfg)

backend: PyTorch
boolx: bool
data:
  adj_transform: normalize_adj
  attr_transform: None
  device: None
  edge_transform: None
  graph_transform: None
  label_transform: None
device: cuda
evaluate:
  Progbar:
    width: 20
  cache_test_data: False
  verbose: 1
fit:
  EarlyStopping:
    baseline: None
    enabled: False
    mode: auto
    monitor: val_loss
    patience: 10
    restore_best_weights: True
    verbose: 1
  Logger:
    enabled: False
    filepath: None
    name: None
  ModelCheckpoint:
    enabled: True
    monitor: val_accuracy
    path: ./GCN_checkpoint_5c41d3.h5
    remove_weights: True
    save_best_only: True
    save_weights_only: True
    vervose: 0
  Progbar:
    width: 20
  TensorBoard:
    enabled: False
    histogram_freq: 0
    log_dir: ./logs
    update_freq: epoch
    write_graph: True
    write_images: True
  TerminateOnNaN:
    enabled: False
  cache_train_data: False
  cache_val_data: False
  epochs: 100
  verbose: 1
floatx: float32
intx: int64
model:
  act